### Import and Define

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
resize = 1
plt.rcParams.update({
    "figure.figsize": (6.4*resize, 4.0*resize), # (6.4, 4.8)[4:3] -> (6.4, 4.0)[8:5]
    "font.sans-serif": ["Helvetica", "Nimbus Sans", "Arial", "DejaVu Sans"],
})

In [ ]:
def restruct_array(content):
    # concatenate all np.ndarray to an array
    time = np.reshape((content.item()['push_time']-content.item()['start_time'])[1:, :], (1, -1))
    train_loss = np.reshape(content.item()['train_loss'][1:, :], (1, -1))
    train_acc = np.reshape(content.item()['train_acc'][1:, :], (1, -1))
    test_loss = np.reshape(content.item()['test_loss'][1:, :], (1, -1))
    test_acc = np.reshape(content.item()['test_acc'][1:, :], (1, -1))
    alles = np.concatenate((time, train_loss, train_acc, test_loss, test_acc))
    # sort the array by push time
    index = np.argsort(alles[0])
    alles = alles[:, index]
    # remove items unused
    index2 = alles[0, :] > 0
    alles = alles[:, index2]
    print(np.min(alles[3]), np.argmin(alles[3]))
    print(alles[3,-1], alles[4,-1])
    return alles

### Load Data

In [ ]:
dir_path = '../conference_IEEE_COMPSAC_2022/scp/tf_npy'
base = '500'
extra = '1.1'
ls = [1, 2, 3]
small = [38, 87, 127]
with_w = []
with_sqrt_w = []
without_w = []

for num1, num2 in zip(ls, small):
    with_w.append(restruct_array(
        np.load(f'./{dir_path}/tf_extra{extra}_{num1}s_{num2}_{base}.npy', allow_pickle=True)
    ))
    with_sqrt_w.append(restruct_array(
        np.load(f'./{dir_path}/tf_sqrt_extra{extra}_{num1}s_{num2}_{base}.npy', allow_pickle=True)
    ))
    without_w.append(restruct_array(
        np.load(f'./{dir_path}/tf_nw_extra{extra}_{num1}s_{num2}_{base}.npy', allow_pickle=True)
    ))

In [ ]:
def get_csv():
    loss = []
    acc = []
    loss.append(np.arange(1, 561)/4)
    acc.append(np.arange(1, 561)/4)
    for i, j, k in zip(with_w, with_sqrt_w, without_w):
        loss.append(i[3])
        loss.append(j[3])
        loss.append(k[3])
        acc.append(i[4])
        acc.append(j[4])
        acc.append(k[4])
    loss = np.array(loss).T
    acc = np.array(acc).T
    np.savetxt('loss_MUfactor.csv', loss, delimiter=',')
    np.savetxt('acc_MUfactor.csv', acc, delimiter=',')

#get_csv()

### Plot

In [ ]:
with_label = []
with_sqrt_label = []
without_label = []

for num1, num2 in zip(ls, small):
    with_label.append(f'$n_S = {num1}, d_S / d_L$')
    with_sqrt_label.append((f'$n_S = {num1}$' + '$, \sqrt{d_S / d_L}$'))
    without_label.append(f'$n_S = {num1}, -$')

#zorder=len(label)-i

In [ ]:
#DPI = 300
SAVE = True
save_path = './figure'

In [ ]:
## testing loss
plt.figure()#(dpi=DPI)
for i in range(len(with_label)):
    plt.plot(np.arange(1, 561)/4, with_w[i][3], label=with_label[i])#, linewidth=1)
    plt.plot(np.arange(1, 561)/4, with_sqrt_w[i][3], '--', label=with_sqrt_label[i])#, linewidth=1)
    plt.plot(np.arange(1, 561)/4, without_w[i][3], ':', label=without_label[i])#, linewidth=1)
plt.ylim(1, 5)
#plt.title('Decaying Learning Rate, Testing Loss vs. Time')
plt.xlabel('Epoch')
plt.ylabel('Testing Loss')
plt.legend()
if SAVE:
    plt.savefig(f"{save_path}/loss_MUfactor.pdf", bbox_inches="tight")
else:
    plt.show()

## testing accuracy
plt.figure()#(dpi=DPI)
for i in range(len(with_label)):
    plt.plot(np.arange(1, 561)/4, with_w[i][4], label=with_label[i])#, linewidth=1)
    plt.plot(np.arange(1, 561)/4, with_sqrt_w[i][4], '--', label=with_sqrt_label[i])#, linewidth=1)
    plt.plot(np.arange(1, 561)/4, without_w[i][4], ':', label=without_label[i])#, linewidth=1)
#plt.title('Decaying Learning Rate, Testing Accuracy vs. Time')
plt.xlabel('Epoch')
plt.ylabel('Testing Accuracy')
plt.legend()
if SAVE:
    plt.savefig(f"{save_path}/acc_MUfactor.pdf", bbox_inches="tight")
else:
    plt.show()